## Chapter 4 Text versus Bytes (2)

### Normalizing unicode for saner comparisons


String 간 비교는 Unicode가 성조와 같은 발음 구별 부호 문자를 갖기 때문에 다소 복잡하다. 이 부호는 앞서 나오는 문자에 붙어 하나의 글자로 보이는 특징(combining characters)을 갖는다. 

In [2]:
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [3]:
len(s1), len(s2)

(4, 5)

In [4]:
s1 == s2

False

위의 예시에서 `s1`과 `s2`의 결과는 같아보이지만, 파이썬은 이 둘을 다른 것으로 본다. 이러한 경우를 처리하기 위해 크게 3가지의 방법이 존재한다. 
1. `unicodedata.normalize(form, unistr)`
2. `str.casefold()`

__unicodedata.normalize(form, unistr)__  

유니코드 문자열 `unistr`에 대한 정규화 형식(`form`)을 반환한다. 
- form : 'NFC', 'NFD', 'NFKC', 'NFKD' 중 하나의 값을 갖는다. 
  - Normalized Form C (NFC), Normalized Form D (NFD): 'canonical equivalance'를 위한 조정, NFD는 정준 분해라고 하며, 각 문자를 분해된 형식(base characters + combining characters)으로 변환한다. NFC는 정준 분해 적용 후, 미리 결합한 문자로 다시 조합(shortest equivalent string)한다. 
  - NFKC, NFKD: 'compatibility equivalence'를 기반으로 하는 정규화 형식. NFKD는 호환 분해를 적용한다. 즉, 모든 호환 문자를 동등한 것으로 치환한다. NFKC는 먼저 호환 분해를 적용한 후, 정준 결합을 적용한다. 

canonical equivalence 예시

In [47]:
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301'
len(s1), len(s2)

(4, 5)

In [53]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [54]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [55]:
normalize('NFC', s1) == normalize('NFC', s2)

True

In [56]:
normalize('NFD', s1) == normalize('NFD', s2)

True

compatibility equivalence 예시

In [71]:
from unicodedata import normalize, name
half = '½'
normalize('NFKC', half)

'1⁄2'

In [75]:
micro = 'µ'
micro_kc = normalize('NFKC', micro)
micro, micro_kc

('µ', 'μ')

In [76]:
ord(micro), ord(micro_kc)

(181, 956)

In [77]:
name(micro), name(micro_kc)

('MICRO SIGN', 'GREEK SMALL LETTER MU')

NFKC나 NFKD는 정보를 왜곡시킬 수도 있다. 예를 들어 4의 제곱을 `42`로 출력한다. 하지만 검색할 때, 유용하게 쓰일 수 있다. (데이터를 변형시킬 수 있기 때문에, 저장하는 용도로는 사용하지 않는 것이 좋다.) 검색할 때 유용하게 쓰이는 연산이 하나 더 있는데, 그게 'Case Folding'이다. 

__str.casefold()__  
  
Case Folding은 모든 문자를 소문자로 바꿔주는 것이다. 

In [92]:
micro = 'µ'
micro_cf = micro.casefold()
micro, micro_cf

('µ', 'μ')

In [93]:
name(micro), name(micro_cf)

('MICRO SIGN', 'GREEK SMALL LETTER MU')

In [94]:
eszett = 'ß'
eszett_cf = eszett.casefold()
eszett, eszett_cf

('ß', 'ss')

이외에도 사람들이 발음 구별 부호를 사용하는 것을 귀찮아하거나, 까먹는다는 사실을 근거로 문자에서 그 부호를 아예 제거하여 검색하는 방법을 사용하기도 한다. 책에서는 이와 관련된 함수를 소개하지만, 여기서는 쓰인 함수에 대한 소개와 예시를 정리하려고 한다. 

In [97]:
import unicodedata

s1 = 'café'
s1_norm = normalize('NFD', s1)
unicodedata.combining(s1_norm[3]), unicodedata.combining(s1_norm[4])

(0, 230)

In [125]:
shift_two = str.maketrans('abcdefghijklmnopqrstuvwxyz',
                          'cdefghijklmnopqrstuvwxyzab')
before = 'unicode'
after = before.translate(shift_two)
after

'wpkeqfg'

### Sorting unicode text

Python에서는 string을 정렬할 때, code point를 비교한다. 이러한 경우 다음 예시와 같이 ASCII 문자 외의 문자에 대하여 이상한 결과를 낳을 수 있다. 

In [126]:
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits)

['acerola', 'açaí', 'atemoia', 'cajá', 'caju']

이를 해결하기 위한 일반적인 방법은 `locale.strxfrm` 함수를 이용하는 것이다. 이 함수를 적절하게 사용하기 위해서는 적절한 지역, 언어를 파악하고 운영체제가 이것을 지원하기를 기도(?)하는 것이다. 

In [127]:
import locale
locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [128]:
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits, key=locale.strxfrm)

['acerola', 'açaí', 'atemoia', 'cajá', 'caju']

원하는 결과가 안 나온다. 

locale.setdefault() 사용 시 주의사항
1. locale setting은 global하게 작용하니, 처음 설정 후 바꾸지 말 것
2. locale이 OS에 설치되어있어야 한다.
3. locale name을 정확히 알고 있어야 한다. 표준 형태가 존재하는데 운영체제마다 형태가 다르다. 
4. locale이 OS 내에서 정상적으로 작동해야 한다. (?)

위의 방법은 GNU/Linux 기반에서만 잘 작동하는 것처럼 보인다. 다행히, 더 간단한 해결방법이 있는데 Unicode Collation Algorithm (UCA)를 이용하는 것이다. 

In [131]:
pip install pyuca

Note: you may need to restart the kernel to use updated packages.


In [132]:
import pyuca
coll = pyuca.Collator()
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits, key=coll.sort_key)

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

### The unicode database

unicode 표준은 database를 제공하는데, 이것은 문자의 code point와 이름을 연결하는 table이나 각각의 문자에 대한 메타정보를 담고 있는 table 등을 포함하고 있고, 이를 확인할 수 있는 코드를 가진다.

In [135]:
# 만약 문자열이 알파벳, 숫자, 기호, space만으로 이루어져있다면 True 반환
a = '\u00bc'
b = '\u0014'
a.isprintable(), b.isprintable()

(True, False)

In [145]:
# 10진수로 구성되어있는지 확인
c = '345'
d = '0b11000'
c.isdecimal(), d.isdecimal()

(True, False)

In [150]:
# 각 문자가 갖는 숫자값을 반환
'\u216b', unicodedata.numeric('\u216b')

('Ⅻ', 12.0)

### Dual-Mode str and bytes APIs

`re`와 `os` 모듈은 str과 bytes 모두를 인자로 받을 수 있고, 어떤 타입이 들어오는지에 따라 다르게 작동한다. 

In [151]:
import re
re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = "Ramanujan saw \u0be7\u0bed\u0be8\u0bef as 1729"
text_bytes = text_str.encode('utf-8')

print('Text', repr(text_str), sep='\n ')
print('Numbers')
print(' str :', re_numbers_str.findall(text_str))
print(' bytes:', re_numbers_bytes.findall(text_bytes))
print('Words')
print(' str :', re_words_str.findall(text_str))
print(' bytes:', re_words_bytes.findall(text_bytes))

Text
 'Ramanujan saw ௧௭௨௯ as 1729'
Numbers
 str : ['௧௭௨௯', '1729']
 bytes: [b'1729']
Words
 str : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729']
 bytes: [b'Ramanujan', b'saw', b'as', b'1729']


ASCII 외의 문자는 bytes에서 숫자로도, 단어로도 취급되지 않는다. 

GNU/Linux kernel은 unicode를 잘 다루지 못한다. 따라서 파일 명에 str로 decode될 수 없는 문자가 있을 경우 문제를 발생시킬 수 있다. 이 문제를 처리하기 위해 모든 os 모듈 내의 함수는 str과 bytes를 모두 인자로 받을 수 있다. 

In [154]:
import os
os.listdir('.'), os.listdir(b'.')

(['Untitled.ipynb',
  'digits-of-π.txt',
  'chapter03-1_jinny.ipynb',
  '.ipynb_checkpoints'],
 [b'Untitled.ipynb',
  b'digits-of-\xcf\x80.txt',
  b'chapter03-1_jinny.ipynb',
  b'.ipynb_checkpoints'])

References  
[unicodedata 문서](https://docs.python.org/ko/3/library/unicodedata.html)  
[locale 문서](https://docs.python.org/3/library/locale.html?highlight=strxfrm#locale.strxfrm)